In [1]:
#default_exp learner

In [2]:
#export
from mantisshrimp.all import *
from torchvision.models.detection.mask_rcnn import MaskRCNNPredictor
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor

# Learner
> Core learner stuff

In [3]:
#export
def rcnn_splitter(m): return L(m.backbone, m.rpn, m.roi_heads).map(params)

In [4]:
#export
def mock_loss(*args, **kwargs): return tensor(0.)

In [5]:
#export
def _get_model(arch, n_class, pretrained=True, h=256):
    model = arch(pretrained)
    # For the classifier
    in_feats = model.roi_heads.box_predictor.cls_score.in_features
    model.roi_heads.box_predictor = FastRCNNPredictor(in_feats, n_class)
    # For the mask
    in_chnns = model.roi_heads.mask_predictor.conv5_mask.in_channels
    model.roi_heads.mask_predictor = MaskRCNNPredictor(in_chnns, 256, n_class)
    return model

In [6]:
#export
@delegates(Learner.__init__)
def mask_rcnn_learner(dls, arch, n_class, pretrained=True, cbs=None, **kwargs):
    model = _get_model(arch, n_class, pretrained=pretrained)
    cbs = L(RCNNCallback) + L(cbs)
    learn = Learner(dls, model, loss_func=mock_loss, splitter=rcnn_splitter, cbs=cbs, **kwargs)
    learn.inference = False
    if pretrained: learn.freeze()
    return learn

## Export -

In [7]:
from nbdev.export import notebook2script
notebook2script()

Converted 00_core.ipynb.
Converted 03_data.load.ipynb.
Converted 04_data.core.ipynb.
Converted 06_callback.ipynb.
Converted 08_learner.ipynb.
Converted index.ipynb.
